## Vytvorenie modelu logistickej regresie - Lekcia 4

![Infografika: Logistická vs. lineárna regresia](../../../../../../2-Regression/4-Logistic/images/linear-vs-logistic.png)

#### **[Kvíz pred prednáškou](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Úvod

V tejto záverečnej lekcii o regresii, jednej zo základných *klasických* techník strojového učenia, sa pozrieme na logistickú regresiu. Túto techniku by ste použili na objavenie vzorcov na predpovedanie binárnych kategórií. Je táto cukrovinka čokoláda alebo nie? Je táto choroba nákazlivá alebo nie? Vyberie si tento zákazník tento produkt alebo nie?

V tejto lekcii sa naučíte:

-   Techniky logistickej regresie

✅ Prehĺbte si svoje porozumenie práce s týmto typom regresie v tomto [učebnom module](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Predpoklady

Keďže sme pracovali s údajmi o tekviciach, sme už dostatočne oboznámení s tým, že existuje jedna binárna kategória, s ktorou môžeme pracovať: `Color`.

Vytvorme model logistickej regresie na predpovedanie toho, *akú farbu bude mať daná tekvica* (oranžová 🎃 alebo biela 👻).

> Prečo hovoríme o binárnej klasifikácii v lekcii zameranej na regresiu? Len z jazykového pohodlia, keďže logistická regresia je [v skutočnosti metóda klasifikácie](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), hoci založená na lineárnom prístupe. O ďalších spôsoboch klasifikácie údajov sa dozviete v nasledujúcej skupine lekcií.

Pre túto lekciu budeme potrebovať nasledujúce balíky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekcia balíkov pre R](https://www.tidyverse.org/packages), ktorá robí dátovú vedu rýchlejšou, jednoduchšou a zábavnejšou!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) je rámec [kolekcie balíkov](https://www.tidymodels.org/packages/) na modelovanie a strojové učenie.

-   `janitor`: Balík [janitor](https://github.com/sfirke/janitor) poskytuje jednoduché nástroje na skúmanie a čistenie nečistých údajov.

-   `ggbeeswarm`: Balík [ggbeeswarm](https://github.com/eclarke/ggbeeswarm) poskytuje metódy na vytváranie grafov v štýle "beeswarm" pomocou ggplot2.

Môžete ich nainštalovať pomocou:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Alternatívne, skript nižšie skontroluje, či máte balíky potrebné na dokončenie tohto modulu, a nainštaluje ich za vás, ak chýbajú.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Definovanie otázky**

Pre naše účely vyjadríme otázku ako binárnu: 'Biela' alebo 'Nie biela'. V našej dátovej sade sa nachádza aj kategória 'pruhovaná', ale má len málo záznamov, takže ju nebudeme používať. Táto kategória aj tak zmizne, keď odstránime nulové hodnoty z dátovej sady.

> 🎃 Zaujímavosť: biele tekvice niekedy nazývame 'duchové' tekvice. Nie sú veľmi ľahké na vyrezávanie, takže nie sú také populárne ako oranžové, ale vyzerajú zaujímavo! Mohli by sme teda našu otázku preformulovať ako: 'Duch' alebo 'Nie duch'. 👻

## **O logistickej regresii**

Logistická regresia sa v niekoľkých dôležitých aspektoch líši od lineárnej regresie, ktorú ste sa učili predtým.

#### **Binárna klasifikácia**

Logistická regresia neponúka rovnaké funkcie ako lineárna regresia. Prvá ponúka predikciu o `binárnej kategórii` ("oranžová alebo nie oranžová"), zatiaľ čo druhá dokáže predpovedať `kontinuálne hodnoty`, napríklad na základe pôvodu tekvice a času zberu *o koľko sa zvýši jej cena*.

![Infografika od Dasani Madipalli](../../../../../../2-Regression/4-Logistic/images/pumpkin-classifier.png)

### Iné typy klasifikácií

Existujú aj iné typy logistickej regresie, vrátane multinomiálnej a ordinálnej:

- **Multinomiálna**, ktorá zahŕňa viac ako jednu kategóriu - "Oranžová, Biela a Pruhovaná".

- **Ordinálna**, ktorá zahŕňa usporiadané kategórie, užitočné, ak chceme logicky zoradiť naše výsledky, napríklad naše tekvice zoradené podľa konečného počtu veľkostí (mini, malá, stredná, veľká, XL, XXL).

![Multinomiálna vs ordinálna regresia](../../../../../../2-Regression/4-Logistic/images/multinomial-vs-ordinal.png)

#### **Premenné NEMUSIA korelovať**

Pamätáte si, ako lineárna regresia fungovala lepšie s viac korelovanými premennými? Logistická regresia je opakom - premenné nemusia byť v súlade. To je výhodné pre tieto dáta, ktoré majú pomerne slabé korelácie.

#### **Potrebujete veľa čistých dát**

Logistická regresia poskytne presnejšie výsledky, ak použijete viac dát; naša malá dátová sada nie je pre túto úlohu optimálna, takže to majte na pamäti.

✅ Zamyslite sa nad typmi dát, ktoré by sa dobre hodili na logistickú regresiu.

## Cvičenie - upravte dáta

Najprv trochu upravte dáta, odstráňte nulové hodnoty a vyberte len niektoré stĺpce:

1. Pridajte nasledujúci kód:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Vždy sa môžete pozrieť na svoj nový dataframe pomocou funkcie [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html), ako je uvedené nižšie:


In [ ]:
pumpkins_select %>% 
  glimpse()


Poďme si potvrdiť, že sa skutočne budeme zaoberať problémom binárnej klasifikácie:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Vizualizácia - kategóriálny graf
Teraz ste znova načítali údaje o tekviciach a vyčistili ich tak, aby ste zachovali dataset obsahujúci niekoľko premenných, vrátane farby (Color). Poďme si vizualizovať dataframe v notebooku pomocou knižnice ggplot.

Knižnica ggplot ponúka niekoľko skvelých spôsobov, ako vizualizovať vaše údaje. Napríklad môžete porovnať rozdelenia údajov pre každú odrodu (Variety) a farbu (Color) v kategóriálnom grafe.

1. Vytvorte takýto graf pomocou funkcie geombar, pričom použijete naše údaje o tekviciach a špecifikujete farebné mapovanie pre každú kategóriu tekvíc (oranžová alebo biela):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

Pozorovaním údajov môžete vidieť, ako sa údaje o Farbe vzťahujú k Odrode.

✅ Na základe tohto kategóriálneho grafu, aké zaujímavé skúmania si dokážete predstaviť?


### Predspracovanie údajov: kódovanie vlastností

Náš dataset tekvíc obsahuje textové hodnoty vo všetkých svojich stĺpcoch. Práca s kategorizovanými údajmi je pre ľudí intuitívna, ale pre stroje nie. Algoritmy strojového učenia pracujú dobre s číslami. Preto je kódovanie veľmi dôležitým krokom vo fáze predspracovania údajov, pretože nám umožňuje premeniť kategorizované údaje na číselné údaje bez straty informácií. Dobré kódovanie vedie k vytvoreniu dobrého modelu.

Pre kódovanie vlastností existujú dva hlavné typy kóderov:

1. Ordinálny kóder: je vhodný pre ordinálne premenné, čo sú kategorizované premenné, kde ich údaje nasledujú logické poradie, ako napríklad stĺpec `item_size` v našom datasete. Vytvára mapovanie, kde každá kategória je reprezentovaná číslom, ktoré zodpovedá poradiu kategórie v stĺpci.

2. Kategorizovaný kóder: je vhodný pre nominálne premenné, čo sú kategorizované premenné, kde ich údaje nenasledujú logické poradie, ako všetky vlastnosti okrem `item_size` v našom datasete. Ide o kódovanie typu one-hot, čo znamená, že každá kategória je reprezentovaná binárnym stĺpcom: kódovaná premenná je rovná 1, ak tekvica patrí do danej Variety, a 0 v opačnom prípade.

Tidymodels poskytuje ďalší šikovný balík: [recipes](https://recipes.tidymodels.org/) - balík na predspracovanie údajov. Definujeme `recipe`, ktorý špecifikuje, že všetky stĺpce prediktorov by mali byť kódované do množiny celých čísel, `prep` na odhad potrebných množstiev a štatistík potrebných pre akékoľvek operácie a nakoniec `bake` na aplikáciu výpočtov na nové údaje.

> Bežne sa recipes zvyčajne používa ako predspracovateľ pre modelovanie, kde definuje, aké kroky by sa mali aplikovať na dataset, aby bol pripravený na modelovanie. V takom prípade je **veľmi odporúčané**, aby ste použili `workflow()` namiesto manuálneho odhadu receptu pomocou prep a bake. Všetko toto si ukážeme už čoskoro.
>
> Avšak momentálne používame recipes + prep + bake na špecifikáciu krokov, ktoré by sa mali aplikovať na dataset, aby bol pripravený na analýzu údajov, a následne extrahujeme predspracované údaje s aplikovanými krokmi.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Aké sú výhody použitia ordinálneho enkodéra pre stĺpec Item Size?

### Analyzujte vzťahy medzi premennými

Teraz, keď sme predspracovali naše údaje, môžeme analyzovať vzťahy medzi vlastnosťami a štítkom, aby sme získali predstavu o tom, ako dobre bude model schopný predpovedať štítok na základe vlastností. Najlepší spôsob, ako vykonať tento typ analýzy, je vizualizácia údajov. 
Opäť použijeme funkciu ggplot geom_boxplot_, aby sme zobrazili vzťahy medzi Item Size, Variety a Color v kategóriálnom grafe. Na lepšiu vizualizáciu údajov použijeme enkódovaný stĺpec Item Size a neenkódovaný stĺpec Variety.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Použitie swarm grafu

Keďže Color je binárna kategória (Biela alebo Nie), vyžaduje si '[špeciálny prístup](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)' k vizualizácii.

Vyskúšajte `swarm graf`, aby ste zobrazili rozloženie farby vo vzťahu k item_size.

Použijeme balík [ggbeeswarm](https://github.com/eclarke/ggbeeswarm), ktorý poskytuje metódy na vytváranie grafov v štýle beeswarm pomocou ggplot2. Beeswarm grafy sú spôsob, ako zobrazovať body, ktoré by sa za normálnych okolností prekrývali, tak, aby boli vedľa seba.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Teraz, keď máme predstavu o vzťahu medzi binárnymi kategóriami farieb a väčšou skupinou veľkostí, poďme preskúmať logistickú regresiu na určenie pravdepodobnej farby danej tekvice.

## Vytvorte svoj model

Vyberte premenné, ktoré chcete použiť vo svojom klasifikačnom modeli, a rozdeľte údaje na tréningovú a testovaciu množinu. [rsample](https://rsample.tidymodels.org/), balík v rámci Tidymodels, poskytuje infraštruktúru na efektívne delenie a resamplovanie údajov:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Teraz sme pripravení trénovať model prispôsobením tréningových vlastností tréningovej značke (farba).

Začneme vytvorením receptu, ktorý špecifikuje kroky predspracovania, ktoré by sa mali vykonať na našich údajoch, aby boli pripravené na modelovanie, t.j. kódovanie kategóriálnych premenných na množinu celých čísel. Rovnako ako `baked_pumpkins`, vytvoríme `pumpkins_recipe`, ale nebudeme používať `prep` a `bake`, pretože to bude zahrnuté do pracovného toku, čo uvidíte o pár krokov neskôr.

Existuje pomerne veľa spôsobov, ako špecifikovať logistickú regresiu v Tidymodels. Pozrite si `?logistic_reg()`. Zatiaľ špecifikujeme logistickú regresiu prostredníctvom predvoleného enginu `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Teraz, keď máme recept a špecifikáciu modelu, musíme nájsť spôsob, ako ich spojiť do objektu, ktorý najskôr predspracuje údaje (príprava + pečenie na pozadí), prispôsobí model na predspracovaných údajoch a zároveň umožní potenciálne aktivity po spracovaní.

V Tidymodels sa tento praktický objekt nazýva [`workflow`](https://workflows.tidymodels.org/) a pohodlne uchováva vaše modelovacie komponenty.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


Po definovaní *pracovného postupu* môže byť model `natrénovaný` pomocou funkcie [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Pracovný postup odhadne recept a pred spracovaním údajov ich predpripraví, takže to nebudeme musieť robiť manuálne pomocou funkcií prep a bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Model vytlačí koeficienty naučené počas tréningu.

Teraz, keď sme model natrénovali pomocou tréningových dát, môžeme robiť predpovede na testovacích dátach pomocou [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Začnime tým, že použijeme model na predpovedanie štítkov pre náš testovací súbor a pravdepodobností pre každý štítok. Keď je pravdepodobnosť vyššia ako 0.5, predpovedaná trieda je `WHITE`, inak `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Veľmi pekné! Toto poskytuje ďalší pohľad na to, ako funguje logistická regresia.

### Lepšie pochopenie pomocou matice zámien

Porovnávanie každej predikcie s jej zodpovedajúcou „skutočnou hodnotou“ nie je veľmi efektívny spôsob, ako určiť, ako dobre model predikuje. Našťastie má Tidymodels v rukáve ešte niekoľko trikov: [`yardstick`](https://yardstick.tidymodels.org/) - balík používaný na meranie efektívnosti modelov pomocou metrík výkonnosti.

Jednou z metrík výkonnosti spojených s klasifikačnými problémami je [`matica zámien`](https://wikipedia.org/wiki/Confusion_matrix). Matica zámien opisuje, ako dobre klasifikačný model funguje. Matica zámien zaznamenáva, koľko príkladov v každej triede bolo modelom správne klasifikovaných. V našom prípade vám ukáže, koľko oranžových tekvíc bolo klasifikovaných ako oranžové a koľko bielych tekvíc bolo klasifikovaných ako biele; matica zámien vám tiež ukáže, koľko ich bolo klasifikovaných do **nesprávnych** kategórií.

Funkcia [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) z yardstick vypočíta túto krížovú tabuľku pozorovaných a predikovaných tried.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Poďme interpretovať maticu zámien. Náš model má za úlohu klasifikovať tekvice do dvoch binárnych kategórií, kategórie `biela` a kategórie `nie-biela`.

-   Ak váš model predpovedá tekvicu ako bielu a v skutočnosti patrí do kategórie 'biela', nazývame to `pravý pozitívny`, čo je znázornené číslom v ľavom hornom rohu.

-   Ak váš model predpovedá tekvicu ako nie-bielu a v skutočnosti patrí do kategórie 'biela', nazývame to `falošný negatívny`, čo je znázornené číslom v ľavom dolnom rohu.

-   Ak váš model predpovedá tekvicu ako bielu a v skutočnosti patrí do kategórie 'nie-biela', nazývame to `falošný pozitívny`, čo je znázornené číslom v pravom hornom rohu.

-   Ak váš model predpovedá tekvicu ako nie-bielu a v skutočnosti patrí do kategórie 'nie-biela', nazývame to `pravý negatívny`, čo je znázornené číslom v pravom dolnom rohu.

| Pravda |
|:-----:|


|               |        |       |
|---------------|--------|-------|
| **Predikcia** | BIELA | ORANŽOVÁ |
| BIELA         | TP     | FP    |
| ORANŽOVÁ      | FN     | TN    |

Ako ste možno uhádli, je preferované mať vyšší počet pravých pozitívnych a pravých negatívnych a nižší počet falošných pozitívnych a falošných negatívnych, čo naznačuje, že model funguje lepšie.

Matica zámien je užitočná, pretože umožňuje odvodiť ďalšie metriky, ktoré nám môžu pomôcť lepšie vyhodnotiť výkon klasifikačného modelu. Poďme si ich prejsť:

🎓 Presnosť: `TP/(TP + FP)` definovaná ako podiel predpokladaných pozitívnych, ktoré sú skutočne pozitívne. Tiež nazývaná [pozitívna prediktívna hodnota](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Návratnosť: `TP/(TP + FN)` definovaná ako podiel pozitívnych výsledkov z počtu vzoriek, ktoré boli skutočne pozitívne. Tiež známa ako `citlivosť`.

🎓 Špecifickosť: `TN/(TN + FP)` definovaná ako podiel negatívnych výsledkov z počtu vzoriek, ktoré boli skutočne negatívne.

🎓 Presnosť modelu: `TP + TN/(TP + TN + FP + FN)` Percento správne predpovedaných označení pre vzorku.

🎓 F Miera: Vážený priemer presnosti a návratnosti, pričom najlepšia hodnota je 1 a najhoršia je 0.

Poďme vypočítať tieto metriky!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Vizualizácia ROC krivky tohto modelu

Urobme ešte jednu vizualizáciu, aby sme si pozreli tzv. [`ROC krivku`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC krivky sa často používajú na zobrazenie výstupu klasifikátora z hľadiska jeho skutočných vs. falošných pozitívnych výsledkov. ROC krivky zvyčajne zobrazujú `True Positive Rate`/citlivosť na osi Y a `False Positive Rate`/1-špecifickosť na osi X. Preto je dôležitá strmosť krivky a priestor medzi stredovou čiarou a krivkou: chcete krivku, ktorá rýchlo stúpa a prechádza nad čiaru. V našom prípade sú na začiatku falošné pozitívne výsledky, a potom krivka správne stúpa a prechádza nad čiaru.

Nakoniec použime `yardstick::roc_auc()` na výpočet skutočnej plochy pod krivkou (Area Under the Curve). Jedným zo spôsobov interpretácie AUC je pravdepodobnosť, že model hodnotí náhodný pozitívny príklad vyššie ako náhodný negatívny príklad.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Výsledok je približne `0.975`. Keďže AUC sa pohybuje v rozmedzí od 0 do 1, chcete dosiahnuť vysoké skóre, pretože model, ktorý je na 100 % presný vo svojich predpovediach, bude mať AUC rovné 1; v tomto prípade je model *celkom dobrý*.

V budúcich lekciách o klasifikáciách sa naučíte, ako zlepšiť skóre vášho modelu (napríklad riešením problémov s nevyváženými údajmi, ako v tomto prípade).

## 🚀Výzva

Logistická regresia ponúka veľa možností na preskúmanie! Najlepší spôsob, ako sa ju naučiť, je experimentovať. Nájdite dataset, ktorý sa hodí na tento typ analýzy, a vytvorte s ním model. Čo ste sa naučili? tip: skúste [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) pre zaujímavé datasety.

## Prehľad a samostatné štúdium

Prečítajte si prvé strany [tohto článku zo Stanfordu](https://web.stanford.edu/~jurafsky/slp3/5.pdf) o niektorých praktických využitiach logistickej regresie. Zamyslite sa nad úlohami, ktoré sú lepšie prispôsobené jednému alebo druhému typu regresných úloh, ktoré sme doteraz študovali. Čo by fungovalo najlepšie?



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie sa odporúča profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
